In [1]:
import numpy as np
from keras.models import Model
from keras.layers import Input, merge, Convolution3D, MaxPooling3D, UpSampling3D, GlobalAveragePooling3D, Dense, Flatten
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD

from keras_tqdm import TQDMNotebookCallback

Using TensorFlow backend.


In [2]:
X = np.load("/mnt/data/luna16/sets/toy_set_v1_20k.npy")
y = np.zeros((20000,2))
y[:10000,0] = 1
y[10000:,1] = 1

idx = np.random.permutation(X.shape[0])
X, y = X[idx], y[idx]

In [4]:
w, h, d = 32, 32, 32

alpha = 2

def get_test3d():
    inputs = Input((w, h, d, 1))
    sz = 32

    conv3dparams = { 'activation':'relu', 'border_mode':'same' }

    x = Convolution3D(sz, 3, 3, 3, **conv3dparams)(inputs)
    x = Convolution3D(sz, 3, 3, 3, **conv3dparams)(x)
    x = BatchNormalization()(x)
    x = MaxPooling3D(pool_size=(2, 2, 2))(x)

    sz = int(sz * alpha)
    x = Convolution3D(sz, 3, 3, 3, **conv3dparams)(x)
    x = Convolution3D(sz, 3, 3, 3, **conv3dparams)(x)
    x = BatchNormalization()(x)
    x = MaxPooling3D(pool_size=(2, 2, 2))(x)

    sz = int(sz * alpha)
    x = Convolution3D(sz, 3, 3, 3, **conv3dparams)(x)
    x = Convolution3D(sz, 3, 3, 3, **conv3dparams)(x)
    x = BatchNormalization()(x)
    x = MaxPooling3D(pool_size=(2, 2, 2))(x)

    sz = int(sz * alpha)
    x = Convolution3D(sz, 3, 3, 3, **conv3dparams)(x)
    x = Convolution3D(sz, 3, 3, 3, **conv3dparams)(x)
    x = BatchNormalization()(x)
    x = MaxPooling3D(pool_size=(2, 2, 2))(x)

    sz = int(sz * alpha)
    x = Convolution3D(sz, 3, 3, 3, **conv3dparams)(x)
    x = Convolution3D(sz, 3, 3, 3, **conv3dparams)(x)
    x = BatchNormalization()(x)

    x = Convolution3D(sz, 2, 2, 2, activation='relu', border_mode='valid')(x)
    x = Flatten()(x)
    # x = GlobalAveragePooling3D()(x)
    x = Dense(2, activation='sigmoid')(x)

    model = Model(input=inputs, output=x)

    return model

model = get_test3d()
print(model.summary())

sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=sgd)

batch_size=32

model.fit(
    X, 
    y, 
    batch_size=batch_size,
    nb_epoch=10,
    validation_split=0.1,
    verbose=0, callbacks=[TQDMNotebookCallback(leave_inner=True)])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 32, 32, 32, 1) 0                                            
____________________________________________________________________________________________________
convolution3d_12 (Convolution3D) (None, 32, 32, 32, 32 896         input_2[0][0]                    
____________________________________________________________________________________________________
convolution3d_13 (Convolution3D) (None, 32, 32, 32, 32 27680       convolution3d_12[0][0]           
____________________________________________________________________________________________________
batchnormalization_6 (BatchNorma (None, 32, 32, 32, 32 128         convolution3d_13[0][0]           
___________________________________________________________________________________________